In [10]:
import numpy as np
from numpy import sort
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from pandas import DataFrame
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
#from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn import metrics

from sklearn.feature_selection import SelectFromModel

from random import seed
from random import randrange

In [11]:
data = pd.read_csv("C:\\Users\Hannah\Desktop\Research Project\data.csv")
data = data.drop(["Unnamed: 32", "id"],1)

X = data.values[:,2:-1].astype('float64')
X = (X - np.mean(X, axis =0)) /  np.std(X, axis = 0)
X = np.hstack([np.ones((X.shape[0], 1)),X]) 
X = MinMaxScaler().fit_transform(X)
Y = data["diagnosis"].map({'M':1,'B':0})
Y = np.array(Y)

In [12]:
X_select = data[['compactness_mean', 'concavity_mean', 'concave points_mean',
       'radius_se', 'perimeter_se', 'area_se',  'texture_worst',
        'area_worst', 'concave points_worst', 'symmetry_worst']]

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X_select, Y, test_size=0.25, random_state=0)
print(X_train.shape)

(426, 10)


In [14]:
def Sigmoid(z):
    return 1/(1 + np.exp(-z))

In [15]:
def Hypothesis(theta, x):   
    return Sigmoid(x @ theta)

In [16]:
def Cost_Function(X,Y,theta,m):
    hi = Hypothesis(theta, x)
    _y = Y.reshape(-1, 1)
    J = 1/float(m) * np.sum(-_y * np.log(hi) - (1-_y) * np.log(1-hi))
    return J

In [17]:
def Cost_Function_Derivative(X,Y,theta,m,alpha):
    hi = Hypothesis(theta,X)
    _y = Y.reshape(-1, 1)
    J = alpha/float(m) * X.T @ (hi - _y)
    return J

def Gradient_Descent(X,Y,theta,m,alpha):
    new_theta = theta - Cost_Function_Derivative(X,Y,theta,m,alpha)
    return new_theta

In [18]:
def Accuracy2(theta):
    correct = 0
    length = len(X_test)
    prediction = (Hypothesis(theta, X_test) > 0.5) #replaced round to use predictions which are correct is ranging between the Y shape below
    _y = Y_test.reshape(-1, 1)
    correct = prediction == _y
    my_accuracy = (np.sum(correct) / length)*100
    print ('LR Accuracy with scikit-learn selected features: ', my_accuracy, "%")
    
def Logistic_Regression2(X,Y,alpha,theta,num_iters):
    m = len(Y)
    for x in range(num_iters):
        new_theta = Gradient_Descent(X,Y,theta,m,alpha)
        theta = new_theta
        if x % 100 == 0:
            print #('theta: ', theta)    
            print #('cost: ', Cost_Function(X,Y,theta,m))
    Accuracy2(theta)
    print('Selected feature weights:', theta)
    #x = np.linspace(-10, 30, 50)
    #y = -(theta[0] + theta[1]*x)/theta[2]
    #plt.plot(X_train, Y_train)
    #plt.plot(theta)
    #plt.show()
    #idxs_selected = select.get_support(indices=True)
    #print(idxs_selected)
  
ep= .012   #sets initial theta to random non zero numbers +/- .012 specifically is arbitrary

initial_theta = np.random.rand(X_train.shape[1],1) * 2 * ep - ep
alpha = 0.5
iterations = 10000
Logistic_Regression2(X_train,Y_train,alpha,initial_theta,iterations)

C:\Users\Hannah\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


LR Accuracy with scikit-learn selected features:  69.2307692308 %
Selected feature weights: [[ -2.51463351e+00]
 [  3.07649775e+01]
 [  1.35559752e+01]
 [ -6.00474920e+01]
 [ -3.79424367e+02]
 [  8.77385043e+02]
 [ -4.19035874e+03]
 [  1.42016189e+02]
 [  2.01605038e+01]
 [ -6.99737552e+01]]
